# 1. 데이터베이스에서 데이터 로드 (처음 한번만 실행)

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta, time 
import json

with open("./db_config.json", "r") as f:
    config = json.load(f)
    
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(engine_url)
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit()

# --- 데이터베이스에서 데이터 로드하는 함수 ---
def load_table_to_df(table_name, engine):
    """지정된 테이블에서 모든 데이터를 Pandas DataFrame으로 로드합니다."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"'{table_name}' 테이블 로드 완료. {len(df)} 행.")
        # 날짜/시간 컬럼 타입 변환 (필요시)
        if 'game_date' in df.columns:
            df['game_date'] = pd.to_datetime(df['game_date'])
        if 'accident_date' in df.columns:
            df['accident_date'] = pd.to_datetime(df['accident_date'])
        if 'weather_date' in df.columns:
            df['weather_date'] = pd.to_datetime(df['weather_date'])
        # start_time, end_time, weather_time은 문자열로 로드될 수 있으므로, 필요시 time 객체로 변환
        return df
    except Exception as e:
        print(f"'{table_name}' 테이블 로드 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 발생 시 빈 DataFrame 반환

# --- 데이터 로드 ---
stadium_df = load_table_to_df('stadium', engine)
sports_game_df = load_table_to_df('sports_game', engine)
traffic_accident_df = load_table_to_df('traffic_accident', engine)
weather_df = load_table_to_df('weather', engine)

# 데이터베이스 연결 종료
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")

MySQL 데이터베이스에 성공적으로 연결되었습니다.
'stadium' 테이블 로드 완료. 60 행.
'sports_game' 테이블 로드 완료. 3631 행.
'traffic_accident' 테이블 로드 완료. 34032 행.
'weather' 테이블 로드 완료. 450811 행.

데이터베이스 연결이 종료되었습니다.


In [45]:
# 사용자로부터 지역명을 입력받아 데이터 셋 구성
START_DATE = pd.to_datetime("2023-01-01") # 분석 시작일
END_DATE = pd.to_datetime("2024-12-31")   # 분석 종료일
stadium_region = stadium_df['region'].unique().tolist()
TARGET_REGION = None
date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
base_df = pd.DataFrame(columns=[
    'date', 'region', 'accident_count', 'injury_count', 'death_count', 'game_count',
    'sports_types', 'temperature', 'precipitation', 'snow_depth', 'weather_condition',
    'is_post_season', 'is_hometeam_win', 'is_holiday', 'weekday', 'audience',
    'game_start_time', 'game_end_time'
])
final_df = base_df.copy()

In [46]:
import numpy as np
for region in stadium_region:
    temp_df = base_df.copy()
    temp_df = pd.DataFrame({'date': date_range})
    temp_df['region_code'] = region
    
    # 스타디움 정보 가져오기
    if not stadium_df.empty:
        stadiums_in_target_region = stadium_df[stadium_df['region'] == region]
        stadium_codes_in_region = stadiums_in_target_region['stadium_code'].unique().tolist()
        #print(f"\n{TARGET_REGION} 내 경기장 코드: {stadium_codes_in_region}")
    else:
        stadium_codes_in_region = []
        #print(f"\n{TARGET_REGION} 내 경기장 정보 없음 또는 stadium 테이블 로드 실패.")
        
    # 스포츠경기 정보 가져오기
    if not sports_game_df.empty and stadium_codes_in_region:
        games_in_region_df = sports_game_df[sports_game_df['stadium_code'].isin(stadium_codes_in_region)]
        games_in_region_df = games_in_region_df.rename(columns={'game_date': 'date'})
        #print(games_in_region_df)
        if not games_in_region_df.empty:
            games_in_region_df["match_type"] = (
                games_in_region_df["match_type"]
                    .replace({"페넌트레이스": "정규시즌",
                            "순위결정전": "정규시즌",
                            "순위결정정": "정규시즌",   # 오타까지 함께 처리
                            '조별리그' : "정규시즌",
                            "0": "정규시즌"})
                    # ➋ 라운드 표기(1R ~ 33R 등) → 정규시즌
                    .str.replace(r"^\d+R$", "정규시즌", regex=True)
            )
            games_in_region_df["match_type"] = (
                games_in_region_df["match_type"]
                    .replace({'와일드카드':"포스트시즌",
                            '준플레이오프':"포스트시즌", 
                            '플레이오프':"포스트시즌", 
                            '한국시리즈':"포스트시즌",
                            '파이널 라운드A':"포스트시즌",
                            '파이널 라운드B':"포스트시즌",
                            '챔피언결정전':"포스트시즌", 
                            '준결승':"포스트시즌", 
                            '결승':"포스트시즌"})
            )
            game_summary_df = games_in_region_df.groupby('date').agg(
                game_count=('stadium_code', 'count'),
                sports_types_list=('sports_type', lambda x: list(set(x))),
                is_post_season_list=('match_type', lambda x: 1 if any('포스트시즌' in str(mt).lower() for mt in x) else 0)
            ).reset_index()
            
            game_summary_df['sports_types'] = game_summary_df['sports_types_list'].apply(lambda x: ','.join(sorted(x)) if x else '없음')
            game_summary_df['is_post_season'] = game_summary_df['is_post_season_list']
            game_summary_df['game_start_time'] = games_in_region_df['start_time']
            game_summary_df['game_end_time'] = games_in_region_df['end_time']
            game_summary_df['is_hometeam_win'] = games_in_region_df['home_team_win']
            game_summary_df['audience'] = games_in_region_df['audience']
            game_summary_df = game_summary_df[['date', 'game_count', 'sports_types', 'is_post_season', 'game_start_time', 'game_end_time', 'is_hometeam_win', 'audience']]
            temp_df = pd.merge(temp_df, game_summary_df, on='date', how='left')
        else:
            print(f"{region} 내 해당 기간 경기 정보 없음.")
            temp_df['game_count'] = 0
            temp_df['sports_types'] = '없음'
            temp_df['is_post_season'] = 0
    else:
        print("sports_game_df 로드 실패 또는 대상 지역 내 경기장 없음.")
        temp_df['game_count'] = 0
        temp_df['sports_types'] = '없음'
        temp_df['is_post_season'] = 0

    temp_df['game_count'] = temp_df['game_count'].fillna(0).astype(int)
    temp_df['sports_types'] = temp_df['sports_types'].fillna('없음')
    temp_df['is_post_season'] = temp_df['is_post_season'].fillna(0).astype(int)
    
    # 교통사고 데이터 가져오기
    if not traffic_accident_df.empty:
        accidents_in_region_df = traffic_accident_df[traffic_accident_df['region'] == region]
        accidents_in_region_df = accidents_in_region_df.rename(columns={'accident_date': 'date'})
        if not accidents_in_region_df.empty:
            accident_summary_df = accidents_in_region_df.groupby('date').agg(
                accident_count_sum=('accident_count', 'sum'),
                injury_count_sum=('injury_count', 'sum'),
                death_count_sum=('death_count', 'sum')
            ).reset_index()
            accident_summary_df = accident_summary_df.rename(columns={'accident_count_sum': 'accident_count', 'injury_count_sum': 'injury_count', 'death_count_sum': 'death_count'})
            
            temp_df = pd.merge(temp_df, accident_summary_df, on='date', how='left')
        else:
            print(f"{region} 내 해당 기간 교통사고 정보 없음.")
            temp_df['accident_count'] = 0
            temp_df['injury_count'] = 0
            temp_df['death_count'] = 0
    else:
        print("traffic_accident_df 로드 실패.")
        temp_df['accident_count'] = 0
        temp_df['injury_count'] = 0
        temp_df['death_count'] = 0
        
    temp_df['accident_count'] = temp_df['accident_count'].fillna(0).astype(int)
    temp_df['injury_count'] = temp_df['injury_count'].fillna(0).astype(int)
    temp_df['death_count'] = temp_df['death_count'].fillna(0).astype(int)
    
    # 날씨 데이터 가져오기
    if not weather_df.empty:
        mask = weather_df['region'].apply(lambda x: x in region)
        weather_region_df = weather_df[mask]
        weather_region_df = weather_region_df.rename(columns={'weather_date': 'date'})

        if not weather_region_df.empty:
            # 날씨 데이터는 하루에 여러 번 기록될 수 있으므로, 일별 집계 필요
            weather_summary_df = weather_region_df.groupby('date').agg(
                temperature=('temperature', 'mean'),
                precipitation=('precipitation', 'sum'),
                snow_depth=('snow_depth', 'sum'),
                avg_cloud_amount=('cloud_amount', 'mean') # 대표 날씨 상태 추론용
            ).reset_index()

            def get_weather_condition(row):
                if (pd.isna(row['precipitation']) or pd.isna(row['snow_depth'])) and pd.isna(row['avg_cloud_amount']):
                    return '정보없음' # 데이터가 없는 경우
                if row['precipitation'] > 0 or pd.isna(row['snow_depth']) > 0:
                    return '비'
                elif pd.notna(row['avg_cloud_amount']):
                    if row['avg_cloud_amount'] >= 7: # (0-10 기준)
                        return '흐림'
                    elif row['avg_cloud_amount'] >= 3:
                        return '약간흐림' # 또는 '약간흐림' 등
                    else:
                        return '맑음'
                return '정보없음' # 강수량 없고 구름 정보도 없는 경우
                
            weather_summary_df['weather_condition'] = weather_summary_df.apply(get_weather_condition, axis=1)
            weather_summary_df = weather_summary_df[['date', 'temperature', 'precipitation', 'snow_depth', 'weather_condition']]
            temp_df = pd.merge(temp_df, weather_summary_df, on='date', how='left')
        else:
            print(f"{region} 내 해당 기간 날씨 정보 없음.")
            temp_df['temperature'] = np.nan
            temp_df['precipitation'] = np.nan
            temp_df['snow_depth'] = np.nan
            temp_df['weather_condition'] = '정보없음'
    else:
        print("weather_df 로드 실패.")
        temp_df['temperature'] = np.nan
        temp_df['precipitation'] = np.nan
        temp_df['snow_depth'] = np.nan
        temp_df['weather_condition'] = '정보없음'
    
    # 공휴일 및 주말 데이터 가져오기
    weekday_map_kr = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
    temp_df['weekday'] = temp_df['date'].dt.dayofweek.map(weekday_map_kr)
    try:
        import holidays
        # temp_df['date']에서 연도를 뽑아와서 unique 값으로 추출
        kr_holidays = holidays.KR(years=temp_df['date'].dt.year.unique().tolist()) 
        is_statutory_holiday = temp_df['date'].apply(lambda d: d in kr_holidays)
        is_saturday = (temp_df['weekday'] == '토')
        is_sunday = (temp_df['weekday'] == '일')
        # 3. 세 가지 조건을 OR 연산자로 결합하여 is_holiday 컬럼 생성
        # (하나라도 True이면 True -> 1, 모두 False이면 False -> 0)
        temp_df['is_holiday'] = (is_statutory_holiday | is_saturday | is_sunday).astype(int)
    except ImportError:
        print("`holidays` 라이브러리가 설치되지 않았습니다. `pip install holidays`로 설치해주세요. 'is_holiday'는 0으로 처리됩니다.")
        temp_df['is_holiday'] = 0
    except Exception as e:
        print(f"공휴일 정보 처리 중 오류: {e}. 'is_holiday'는 0으로 처리됩니다.")
        temp_df['is_holiday'] = 0

    final_df = pd.merge(final_df,temp_df,how='outer')

경기 안양시 내 해당 기간 교통사고 정보 없음.
경남 창원시 내 해당 기간 교통사고 정보 없음.
경기 김포시 내 해당 기간 교통사고 정보 없음.
경기 고양시 내 해당 기간 교통사고 정보 없음.
경북 포항시 내 해당 기간 날씨 정보 없음.
서울 구로구 내 해당 기간 교통사고 정보 없음.


In [47]:
final_df['date'] = final_df['date'].dt.strftime('%Y-%m-%d')
final_df['temperature'] = pd.to_numeric(final_df['temperature'], errors='coerce').round(1)
final_df['precipitation'] = pd.to_numeric(final_df['precipitation'], errors='coerce').round(1)
final_df['snow_depth'] = pd.to_numeric(final_df['precipitation'], errors='coerce').round(1)

# 4. 최종 데이터 셋 csv 파일로 저장 및 DataFrame 출력 

In [48]:
from datetime import datetime
final_df.to_csv(f"./results/2nd-dataset_전국_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv", index=False)
print("\n--- Final Dataset ---")
final_df


--- Final Dataset ---


,date,region,accident_count,injury_count,death_count,game_count,sports_types,temperature,precipitation,snow_depth,weather_condition,is_post_season,is_hometeam_win,is_holiday,weekday,audience,game_start_time,game_end_time,region_code
0,2023-01-01,NaN,0,0,0,0,없음,-2.5,0.0,0.0,정보없음,0,NaN,1,일,NaN,NaT,NaT,경북 김천시
1,2023-01-01,NaN,0,0,0,0,없음,-2.1,0.0,0.0,맑음,0,NaN,1,일,NaN,NaT,NaT,강원 춘천시
2,2023-01-01,NaN,0,0,0,0,없음,-0.2,0.0,0.0,정보없음,0,NaN,1,일,NaN,NaT,NaT,경기 김포시
3,2023-01-01,NaN,0,0,0,0,없음,0.3,0.0,0.0,맑음,0,NaN,1,일,NaN,NaT,NaT,서울 구로구
4,2023-01-01,NaN,0,0,0,0,없음,0.3,0.0,0.0,맑음,0,NaN,1,일,NaN,NaT,NaT,서울 중구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28504,2024-12-31,NaN,10,12,0,0,없음,0.9,0.0,0.0,맑음,0,NaN,0,화,NaN,NaT,NaT,경기 수원시
28505,2024-12-31,NaN,12,16,0,0,없음,8.6,0.0,0.0,약간흐림,0,NaN,0,화,NaN,NaT,NaT,제주 제주시
28506,2024-12-31,NaN,14,20,0,0,없음,0.3,0.0,0.0,정보없음,0,NaN,0,화,NaN,NaT,NaT,경기 안산시
28507,2024-12-31,NaN,14,22,0,0,없음,0.7,0.0,0.0,정보없음,0,NaN,0,화,NaN,NaT,NaT,경기 화성시
